In [133]:
import pandas as pd
import numpy as np
import panel as pn
import folium as fm
import param
pn.extension('tabulator')
pn.extension(sizing_mode="stretch_width")
import hvplot.pandas

## (1) Define Panel Widgets

In [134]:
# Define Panel widgets
week_slider = pn.widgets.IntSlider(name='Week slider', start=1, end=52, step=1, value=9)
#week_slider

In [135]:
# Define Panel widgets
confidence_slider = pn.widgets.IntSlider(name='Confidence slider', start=50, end=100, step=1, value=75)
confidence_slider

IntSlider(end=100, name='Confidence slider', sizing_mode='stretch_width', start=50, value=75)

In [136]:
confidence_range_slider = pn.widgets.IntRangeSlider(name='Confidence',start=50, end=100, value=(50, 100), step=1)

confidence_range_slider

IntRangeSlider(end=100, name='Confidence', sizing_mode='stretch_width', start=50, value=(50, 100), value_end=100, value_start=50)

In [137]:
confidence_range_slider.value[0]

50

In [138]:
week_range_slider = pn.widgets.IntRangeSlider(name='Week',start=9, end=52, value=(9, 52), step=1)

week_range_slider

IntRangeSlider(end=52, name='Week', sizing_mode='stretch_width', start=9, value=(9, 52), value_end=52, value_start=9)

## (2) Reading Data

In [139]:
#dfm = pd.read_csv('wildfire-data/active-data/ts_active_fire_shape_24h.csv')

In [140]:
#dfm = pd.read_csv('https://raw.githubusercontent.com/oneon/firedataset/main/ts_active_fire_shape.csv')

In [141]:
# cache data to improve dashboard performance
if 'datamap' not in pn.state.cache.keys():

    dfm = pd.read_csv('https://raw.githubusercontent.com/oneon/firedataset/main/ts_active_fire_shape.csv')

    pn.state.cache['datamap'] = dfm.copy()

else: 

    dfm = pn.state.cache['datamap']

In [142]:
# cache data to improve dashboard performance
if 'datapre' not in pn.state.cache.keys():

    dfp = pd.read_csv('https://raw.githubusercontent.com/oneon/firedataset/main/ts_predict_fire_shape.csv')

    pn.state.cache['datapre'] = dfp.copy()

else: 

    dfp = pn.state.cache['datapre']

In [143]:
dfm = dfm[dfm['Dist_Name'].str.contains('Nalgonda') | dfm['Dist_Name'].str.contains('Nagarkurnool')]

In [144]:
dfp = dfp[dfp['Dist_Name'].str.contains('Nalgonda') | dfp['Dist_Name'].str.contains('Nagarkurnool')]

In [145]:
#dfp

In [146]:
dfm.shape, dfp.shape

((249, 13), (373, 13))

In [147]:
# Make DataFrame Pipeline Interactive
idfm = dfm.interactive()

In [148]:
idfm.shape

In [149]:
yaxis_duration = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['24h', '48h', '7d'], 
    button_type='success'
)
yaxis_duration

map_data_pipeline = (
    idfm[
        (idfm.duration == yaxis_duration)
    ]
    .groupby(['latitude', 'longitude', 'acq_date', 'acq_time', 'Mandal_Nam', 'Dist_Name', 'duration', 'instrument', 'week']).confidence.max()
    .to_frame()
    .reset_index()  
    .reset_index(drop=True)
)

In [150]:
#map_data_pipeline.columns

## (3) Table - Firedata 

In [151]:
active_fire_table = map_data_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 20, sizing_mode='stretch_width') 
#active_fire_table

In [152]:
#map_data_pipeline.sample(10)

In [153]:
def get_map(lat=16.385318, long=78.9740939, zoom_start=9):
    return fm.Map(location=[lat,long], zoom_start=zoom_start)

map = get_map()

#pn.panel(map, height=400)

## (4) Data Active - Firedata 

In [154]:
def get_df_aqi():
    aqi = dfm[dfm['duration'] == '24h'] 
    return pd.DataFrame(aqi)
    #return aqi

df_aqi = get_df_aqi()

## (5) Data Forecast - Firedata 

In [155]:
def get_dfp_aqi(confidence=50):
    aqi = dfp[dfp['week'] > 8] 
    aqi = aqi[aqi['week'] < 14]
    aqi = aqi[aqi['confidence'] >= confidence]
    return pd.DataFrame(aqi)
    #return aqi

df_aqii = get_dfp_aqi()

In [156]:
df_aqi.shape, df_aqii.shape

((59, 13), (300, 13))

In [157]:
def add_aqi_circles(map, df_aqi):
    green_p1  = fm.map.FeatureGroup()
    yellow_p1 = fm.map.FeatureGroup()
    orange_p1 = fm.map.FeatureGroup()
    red_p1    = fm.map.FeatureGroup()
    purple_p1 = fm.map.FeatureGroup()
    maroon_p1 = fm.map.FeatureGroup()

    for _, row in df_aqi.iterrows():
        if row.confidence<10:
            feature_group = green_p1
            fill_color = "green"
        elif row.confidence < 30:
            feature_group = yellow_p1
            fill_color = "yellow"
        elif row.confidence < 50:
            feature_group = orange_p1
            fill_color = "orange"
        elif row.confidence < 70:
            feature_group = red_p1
            fill_color = "red"
        elif row.confidence < 90:
            feature_group = purple_p1
            fill_color='purple'
        else:
            feature_group = maroon_p1
            fill_color = "maroon"
    
    
        html=f"""
            <h4> {row.Dist_Name}</h4>
            <p>Mandal {row.Mandal_Nam}</p>
            <ul>
                <li>Date {row.acq_date}</li>
                <li>Time {row.acq_time}</li>
                <li>Lat {row.latitude}</li>
                <li>Long {row.longitude}</li>
                <li>Confidence {row.confidence}</li>
            </ul>
            </p>
            """
        iframe = fm.IFrame(html=html, width=200, height=200)
        popup = fm.Popup(iframe, max_width=2650)
        pushpin = fm.features.CustomIcon('fire-flame.png', icon_size=(30,30))
        feature_group.add_child(
            fm.Marker(
                [row.latitude, row.longitude],
                icon=pushpin,
                popup=popup
            )
        )

    map.add_child(green_p1)
    map.add_child(yellow_p1)
    map.add_child(orange_p1)
    map.add_child(red_p1)
    map.add_child(purple_p1)
    
#add_aqi_circles(map, df_aqi)
#pn.panel(map, height=400)

In [158]:
def add_aqi_predict_circles(map, df_aqi):
    green_p1  = fm.map.FeatureGroup()
    yellow_p1 = fm.map.FeatureGroup()
    orange_p1 = fm.map.FeatureGroup()
    red_p1    = fm.map.FeatureGroup()
    purple_p1 = fm.map.FeatureGroup()
    maroon_p1 = fm.map.FeatureGroup()

    for _, row in df_aqi.iterrows():
        if row.confidence<10:
            feature_group = green_p1
            fill_color = "green"
        elif row.confidence < 30:
            feature_group = yellow_p1
            fill_color = "yellow"
        elif row.confidence < 50:
            feature_group = orange_p1
            fill_color = "orange"
        elif row.confidence < 70:
            feature_group = red_p1
            fill_color = "red"
        elif row.confidence < 90:
            feature_group = purple_p1
            fill_color='purple'
        else:
            feature_group = maroon_p1
            fill_color = "maroon"
    
    
        html=f"""
            <h4> {row.Dist_Name}</h4>
            <p>Mandal {row.Mandal_Nam}</p>
            <ul>
                <li>Week {row.week}</li>
                <li>Lat {row.latitude}</li>
                <li>Long {row.longitude}</li>
                <li>Confidence {row.confidence}</li>
            </ul>
            </p>
            """
        iframe = fm.IFrame(html=html, width=200, height=200)
        popup = fm.Popup(iframe, max_width=2650)
        pushpin = fm.features.CustomIcon('fire-flame.png', icon_size=(30,30))
        feature_group.add_child(
            fm.Marker(
                [row.latitude, row.longitude],
                icon=pushpin,
                popup=popup
            )
        )

    map.add_child(green_p1)
    map.add_child(yellow_p1)
    map.add_child(orange_p1)
    map.add_child(red_p1)
    map.add_child(purple_p1)
    
#add_aqi_predict_circles(map, df_aqii)
#pn.panel(map, height=400)

## (6) Create Map Active Fire 

In [159]:
class PanelFoliumMap(param.Parameterized):
    points_count = param.Integer(20, bounds=(10,100))
        
    def __init__(self, **params):
        super().__init__(**params)
        self.map = get_map()
        self.folium_pane = pn.pane.plot.Folium(sizing_mode="stretch_both", min_height=500, margin=0)    
        self.view = pn.Column(
            self.param.points_count,
            self.folium_pane,
            sizing_mode="stretch_both", height=500
        )
        self._update_map()

    @param.depends("points_count", watch=True)
    def _update_map(self):
        self.map = get_map()
        df_aqi = get_df_aqi()
        add_aqi_circles(self.map, df_aqi)
        self.folium_pane.object = self.map

        
#app = PanelFoliumMap()
#app.view

## (7) Bar chart with Forecast data

In [160]:
dfp

,latitude,longitude,year,week,fire_cnt_before,fire_before,fire_cnt_last_year,fire_last_year,fire_cnt_last_year_same_week,fire_last_year_same_week,Mandal_Nam,Dist_Name,confidence
10,16.02,78.76,2022,8,0.058824,0.019608,0.078431,0.039216,0,0,Amrabad,Nagarkurnool,60.932167
11,16.03,78.71,2022,9,0.058824,0.019608,0.000000,0.000000,0,0,Amrabad,Nagarkurnool,60.932167
12,16.03,78.72,2022,9,0.156863,0.019608,0.000000,0.000000,0,0,Amrabad,Nagarkurnool,88.000000
13,16.03,78.73,2022,9,0.098039,0.019608,0.019608,0.019608,0,0,Amrabad,Nagarkurnool,85.000000
14,16.03,78.76,2022,8,0.078431,0.039216,0.000000,0.000000,0,0,Amrabad,Nagarkurnool,68.886373
...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,17.22,79.03,2022,50,0.294118,0.235294,0.431373,0.235294,0,0,Chityal,Nalgonda,99.970766
592,17.22,79.03,2022,51,0.294118,0.235294,0.431373,0.235294,1,1,Chityal,Nalgonda,99.970766
593,17.22,79.03,2022,52,0.294118,0.235294,0.431373,0.235294,0,0,Chityal,Nalgonda,81.000000
594,17.22,79.05,2022,48,0.058824,0.019608,0.000000,0.000000,0,0,Chityal,Nalgonda,60.586574


In [161]:
# Make DataFrame Pipeline Interactive
idfp = dfp.interactive()

In [162]:
dfp

,latitude,longitude,year,week,fire_cnt_before,fire_before,fire_cnt_last_year,fire_last_year,fire_cnt_last_year_same_week,fire_last_year_same_week,Mandal_Nam,Dist_Name,confidence
10,16.02,78.76,2022,8,0.058824,0.019608,0.078431,0.039216,0,0,Amrabad,Nagarkurnool,60.932167
11,16.03,78.71,2022,9,0.058824,0.019608,0.000000,0.000000,0,0,Amrabad,Nagarkurnool,60.932167
12,16.03,78.72,2022,9,0.156863,0.019608,0.000000,0.000000,0,0,Amrabad,Nagarkurnool,88.000000
13,16.03,78.73,2022,9,0.098039,0.019608,0.019608,0.019608,0,0,Amrabad,Nagarkurnool,85.000000
14,16.03,78.76,2022,8,0.078431,0.039216,0.000000,0.000000,0,0,Amrabad,Nagarkurnool,68.886373
...,...,...,...,...,...,...,...,...,...,...,...,...,...
591,17.22,79.03,2022,50,0.294118,0.235294,0.431373,0.235294,0,0,Chityal,Nalgonda,99.970766
592,17.22,79.03,2022,51,0.294118,0.235294,0.431373,0.235294,1,1,Chityal,Nalgonda,99.970766
593,17.22,79.03,2022,52,0.294118,0.235294,0.431373,0.235294,0,0,Chityal,Nalgonda,81.000000
594,17.22,79.05,2022,48,0.058824,0.019608,0.000000,0.000000,0,0,Chityal,Nalgonda,60.586574


In [163]:
# Radio buttons for District Name
yaxis_district = pn.widgets.RadioButtonGroup(
    name='Y axis', 
    options=['Nagarkurnool', 'Nalgonda',],
    button_type='success'
)
week_predict_bar_pipeline = (
    idfp[
        (idfp.Dist_Name == yaxis_district)       
    ]
    .groupby(['week']).size()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

In [164]:
#week_predict_bar_pipeline

In [165]:
week_predict_bar_pipeline.columns = ['week', 'Dist_Name', 'fire_count']

In [166]:
#week_predict_bar_pipeline

In [167]:
week_predict_bar_plot = week_predict_bar_pipeline.hvplot(kind='bar', 
                                                     x='week', 
                                                     y='0', 
                                                     title='Weekly fire count forecast')
week_predict_bar_plot

## (8) Week vs Confidence scatterplot

In [168]:
week_vs_conf_scatterplot_pipeline = (
    idfp
    .groupby(['latitude', 'longitude', 'week', 'Dist_Name'])['confidence'].mean()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

In [169]:
week_vs_conf_scatterplot = week_vs_conf_scatterplot_pipeline.hvplot(x='week', 
                                                                y='confidence', 
                                                                by='Dist_Name', 
                                                                size=80, kind="scatter", 
                                                                alpha=0.7,
                                                                legend=False, 
                                                                height=500, 
                                                                width=500)
#week_vs_conf_scatterplot

## (9) Table - Firedata Forecast 

In [170]:
week_predict_pipeline = (
    idfp[
        (idfp.week == week_slider)
    ].groupby(['latitude', 'longitude', 'week', 'Dist_Name'])['confidence'].mean()
    .reset_index()
    .reset_index(drop=True)
)

In [171]:
#week_predict_pipeline

In [172]:
predict_table = week_predict_pipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size = 20, sizing_mode='stretch_width') 
#predict_table

## (10) Create Map Forecast 

In [173]:
class PanelFoliumMapForecast(param.Parameterized):
    Confidence = param.Integer(50, bounds=(50,100))
        
    def __init__(self, **params):
        super().__init__(**params)
        self.map = get_map()
        self.folium_pane = pn.pane.plot.Folium(sizing_mode="stretch_both", min_height=500, margin=0)    
        self.view = pn.Column(
            self.param.Confidence,
            self.folium_pane,
            sizing_mode="stretch_both", height=500
        )
        self._update_map()

    @param.depends("Confidence", watch=True)
    def _update_map(self):
        self.map = get_map()
        df_aqi = get_dfp_aqi(confidence=self.Confidence)
        add_aqi_predict_circles(self.map, df_aqi)
        self.folium_pane.object = self.map

        
#app = PanelFoliumMapForecast()
#app.view

## (11) Creating Dashboard

In [187]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Amrabad Wild-Fire Forecast', 
    sidebar=[pn.pane.Markdown("# Forecast Wildfire"), 
             pn.pane.Markdown("# Track Active Wildfire"), 
             #pn.pane.Markdown("#### Wild fire is one of the primary reason for climate change hazard.Track wild fire realtime and forecast wildfire"), 
             #pn.pane.PNG('fire-flame.png', sizing_mode='scale_both'),
             pn.pane.Markdown("### Remote satellites selected for Wild-Fire prediction & detection."),
             pn.pane.Markdown("""
                                | Satellites      |    Instruments |
                                | ----------------| -------------- |
                                | Aqua, Terra     |    MODIS       |
                                | S-NPP, NOAA20   |    VIIRS       |

                                """, style={'border': "4px solid blue"}),
            pn.pane.PNG('fire-flame.png', sizing_mode='scale_both')],
    main=[pn.pane.Markdown("# Wild-Fire Prediction"),
          pn.Row(pn.Column(PanelFoliumMapForecast().view),
                 pn.Column(week_slider,predict_table.panel(width=500))),
          pn.Row(pn.Column(week_vs_conf_scatterplot.panel(width=600), margin=(0,25)), 
                 pn.Column(yaxis_district, week_predict_bar_plot.panel(width=600))),
          pn.pane.Markdown("# Wild-Fire Detection", align="center"),
          pn.Row(pn.Column(yaxis_duration,
                           PanelFoliumMap().view), 
                 active_fire_table.panel(width=500)),
          ],
    accent_base_color="#88d8b0", 
)
#template.show()
template.servable();